<a href="https://colab.research.google.com/github/GrainSack/neural-tangents/blob/main/glocal_k_with_ntk_12_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.9.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.9.2


In [2]:
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==1.15


In [3]:
from time import time
from scipy.sparse import csc_matrix
import tensorflow as tf
import numpy as np
import h5py

In [4]:
print(tf.__version__)

2.9.2


In [5]:
pwd

'/content'

# Data Loader Function

In [3]:
def load_data_100k(path='./content/drive/MyDrive/MovieLens_100K', delimiter='\t'):

    train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [7]:
def load_data_1m(path='./content/drive/MyDrive/MovieLens_1M', delimiter='::', frac=0.1, seed=1234):

    tic = time()
    print('reading data...')
    data = np.loadtxt(path+'movielens_1m_dataset.dat', skiprows=0, delimiter=delimiter).astype('int32')
    print('taken', time() - tic, 'seconds')

    n_u = np.unique(data[:,0]).size  # num of users
    n_m = np.unique(data[:,1]).size  # num of movies
    n_r = data.shape[0]  # num of ratings

    udict = {}
    for i, u in enumerate(np.unique(data[:,0]).tolist()):
        udict[u] = i
    mdict = {}
    for i, m in enumerate(np.unique(data[:,1]).tolist()):
        mdict[m] = i

    np.random.seed(seed)
    idx = np.arange(n_r)
    np.random.shuffle(idx)

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_r):
        u_id = data[idx[i], 0]
        m_id = data[idx[i], 1]
        r = data[idx[i], 2]

        if i < int(frac * n_r):
            test_r[mdict[m_id], udict[u_id]] = r
        else:
            train_r[mdict[m_id], udict[u_id]] = r

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_r - int(frac * n_r)))
    print('num of test ratings: {}'.format(int(frac * n_r)))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [8]:
def load_matlab_file(path_file, name_field):
    
    db = h5py.File(path_file, 'r')
    ds = db[name_field]

    try:
        if 'ir' in ds.keys():
            data = np.asarray(ds['data'])
            ir   = np.asarray(ds['ir'])
            jc   = np.asarray(ds['jc'])
            out  = csc_matrix((data, ir, jc)).astype(np.float32)
    except AttributeError:
        out = np.asarray(ds).astype(np.float32).T

    db.close()

    return out

In [9]:
def load_data_monti(path='./'):

    M = load_matlab_file(path+'douban_monti_dataset.mat', 'M')
    Otraining = load_matlab_file(path+'douban_monti_dataset.mat', 'Otraining') * M
    Otest = load_matlab_file(path+'douban_monti_dataset.mat', 'Otest') * M

    n_u = M.shape[0]  # num of users
    n_m = M.shape[1]  # num of movies
    n_train = Otraining[np.where(Otraining)].size  # num of training ratings
    n_test = Otest[np.where(Otest)].size  # num of test ratings

    train_r = Otraining.T
    test_r = Otest.T

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

# Load Data

In [4]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = '/content/drive/'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [11]:
# Select a dataset among 'ML-1M', 'ML-100K', and 'Douban'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
dataset = 'ML-1M'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [12]:
# Data Load
try:
    if dataset == 'ML-100K':
        path = '/content/drive/MyDrive/MovieLens_100K/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(path=path, delimiter='\t')

    elif dataset == 'ML-1M':
        path = '/content/drive/MyDrive/MovieLens_1M/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_1m(path=path, delimiter='::', frac=0.1, seed=1234)

    elif dataset == 'Douban':
        path = data_path + 'MyDrive/Douban_monti/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_monti(path=path)

    else:
        raise ValueError

except ValueError:
    print('Error: Unable to load data')

reading data...
taken 10.451043844223022 seconds
data matrix loaded
num of users: 6040
num of movies: 3706
num of training ratings: 900189
num of test ratings: 100020


# Hyperparameter Settings

In [13]:
# Common hyperparameter settings
n_hid = 500
n_dim = 5
n_layers = 2
gk_size = 3

In [14]:
# Different hyperparameter settings for each dataset
if dataset == 'ML-100K':
    lambda_2 = 20.  # l2 regularisation
    lambda_s = 0.006
    iter_p = 5  # optimisation
    iter_f = 5
    epoch_p = 30  # training epoch
    epoch_f = 60
    dot_scale = 1  # scaled dot product

elif dataset == 'ML-1M':
    lambda_2 = 70.
    lambda_s = 0.018
    iter_p = 50
    iter_f = 10
    epoch_p = 20
    epoch_f = 30
    dot_scale = 0.5

elif dataset == 'Douban':
    lambda_2 = 10.
    lambda_s = 0.022
    iter_p = 5
    iter_f = 5
    epoch_p = 20
    epoch_f = 60
    dot_scale = 2

In [16]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
R = tf.placeholder("float", [n_m, n_u])

# Network Function

In [18]:
def local_kernel(u, v):

    dist = tf.norm(u - v, ord=2, axis=2)
    hat = tf.maximum(0., 1. - dist**2)

    return hat

In [27]:
import tensorflow as tf


import tensorflow_datasets as tfds

In [57]:
def kernel_layer(x, n_hid=n_hid, n_dim=n_dim, activation=tf.nn.sigmoid, lambda_s=lambda_s, lambda_2=lambda_2, name=''):

    with tf.compat.v1.variable_scope(name, reuse=True):
        W = tf.compat.v1.get_variable('W', [x.shape[1], n_hid])
        n_in = x.get_shape().as_list()[1]
        u = tf.compat.v1.get_variable('u', initializer=tf.random.truncated_normal([n_in, 1, n_dim], 0., 1e-3))
        v = tf.compat.v1.get_variable('v', initializer=tf.random.truncated_normal([1, n_hid, n_dim], 0., 1e-3))
        b = tf.compat.v1.get_variable('b', [n_hid])

    w_hat = local_kernel(u, v)
    
    #sparse_reg = tf.contrib.layers.l2_regularizer(lambda_s)
    #sparse_reg_term = tf.contrib.layers.apply_regularization(sparse_reg, [w_hat])
    #sparse_reg = tf.keras.regularizers.L2(lambda_s)
    sparse_reg_term = tf.keras.regularizers.L2(lambda_s,[w_hat] )
    
    #l2_reg = tf.contrib.layers.l2_regularizer(lambda_2)
    #l2_reg_term = tf.contrib.layers.apply_regularization(l2_reg, [W])
    #l2_reg = tf.keras.regularizers.L2(lambda_2)
    l2_reg_term = tf.keras.regularizers.l2(lambda_s)

    W_eff = W * w_hat  # Local kernelised weight matrix
    y = tf.matmul(x, W_eff) + b
    y = activation(y)
    elfe = sparse_reg_term + l2_reg_term

    return y, elfe
    #return y, sparse_reg_term + l2_reg_term


In [20]:
def global_kernel(input, gk_size, dot_scale):

    avg_pooling = tf.reduce_mean(input, axis=1)  # Item (axis=1) based average pooling
    avg_pooling = tf.reshape(avg_pooling, [1, -1])
    n_kernel = avg_pooling.shape[1].value

    conv_kernel = tf.get_variable('conv_kernel', initializer=tf.random.truncated_normal([n_kernel, gk_size**2], stddev=0.1))
    gk = tf.matmul(avg_pooling, conv_kernel) * dot_scale  # Scaled dot product
    gk = tf.reshape(gk, [gk_size, gk_size, 1, 1])

    return gk

In [21]:
def global_conv(input, W):

    input = tf.reshape(input, [1, input.shape[0], input.shape[1], 1])
    conv2d = tf.nn.relu(tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME'))

    return tf.reshape(conv2d, [conv2d.shape[1], conv2d.shape[2]])

# Network Instantiation

## Pre-training

In [ ]:
(기존) initializer = tf.contrib.layers.xavier_initializer()

# -outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

 

(변경) initializer = tf.truncated_normal_initializer(stddev=0.1)

outputs, states = tf.compat.v1.nn.static_rnn(lstm_cells, _X, dtype=tf.float32)

In [58]:
y = R
reg_losses = None

for i in range(n_layers):
    y, reg_loss = kernel_layer(y, name=str(i))
    reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss

pred_p, reg_loss = kernel_layer(y, n_u, activation=tf.identity, name='out')
reg_losses = reg_losses + reg_loss

# L2 loss
diff = train_m * (train_r - pred_p)
sqE = tf.nn.l2_loss(diff)
loss_p = sqE + reg_losses

optimizer_p = tf.contrib.opt.ScipyOptimizerInterface(loss_p, options={'disp': True, 'maxiter': iter_p, 'maxcor': 10}, method='L-BFGS-B')
#optimizer_p = tf.contrib.opt.ScipyOptimizerInterface(loss_p, options={'disp': True, 'maxiter': iter_p, 'maxcor': 10}, method='L-BFGS-B')

TypeError: ignored

## Fine-tuning

In [ ]:
y = R
reg_losses = None

for i in range(n_layers):
    y, _ = kernel_layer(y, name=str(i))

y_dash, _ = kernel_layer(y, n_u, activation=tf.identity, name='out')

gk = global_kernel(y_dash, gk_size, dot_scale)  # Global kernel
y_hat = global_conv(train_r, gk)  # Global kernel-based rating matrix

for i in range(n_layers):
    y_hat, reg_loss = kernel_layer(y_hat, name=str(i))
    reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss

pred_f, reg_loss = kernel_layer(y_hat, n_u, activation=tf.identity, name='out')
reg_losses = reg_losses + reg_loss

# L2 loss
diff = train_m * (train_r - pred_f)
sqE = tf.nn.l2_loss(diff)
loss_f = sqE + reg_losses

optimizer_f = tf.contrib.opt.ScipyOptimizerInterface(loss_f, options={'disp': True, 'maxiter': iter_f, 'maxcor': 10}, method='L-BFGS-B')

# Evaluation code

In [ ]:
def dcg_k(score_label, k):
    dcg, i = 0., 0
    for s in score_label:
        if i < k:
            dcg += (2**s[1]-1) / np.log2(2+i)
            i += 1
    return dcg

In [ ]:
def ndcg_k(y_hat, y, k):
    score_label = np.stack([y_hat, y], axis=1).tolist()
    score_label = sorted(score_label, key=lambda d:d[0], reverse=True)
    score_label_ = sorted(score_label, key=lambda d:d[1], reverse=True)
    norm, i = 0., 0
    for s in score_label_:
        if i < k:
            norm += (2**s[1]-1) / np.log2(2+i)
            i += 1
    dcg = dcg_k(score_label, k)
    return dcg / norm

In [ ]:
def call_ndcg(y_hat, y):
    ndcg_sum, num = 0, 0
    y_hat, y = y_hat.T, y.T
    n_users = y.shape[0]

    for i in range(n_users):
        y_hat_i = y_hat[i][np.where(y[i])]
        y_i = y[i][np.where(y[i])]

        if y_i.shape[0] < 2:
            continue

        ndcg_sum += ndcg_k(y_hat_i, y_i, y_i.shape[0])  # user-wise calculation
        num += 1

    return ndcg_sum / num

# Training and Test Loop

In [ ]:
best_rmse_ep, best_mae_ep, best_ndcg_ep = 0, 0, 0
best_rmse, best_mae, best_ndcg = float("inf"), float("inf"), 0

time_cumulative = 0
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch_p):
        tic = time()
        optimizer_p.minimize(sess, feed_dict={R: train_r})
        pre = sess.run(pred_p, feed_dict={R: train_r})

        t = time() - tic
        time_cumulative += t
        
        error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
        test_rmse = np.sqrt(error)

        error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
        train_rmse = np.sqrt(error_train)

        print('.-^-._' * 12)
        print('PRE-TRAINING')
        print('Epoch:', i+1, 'test rmse:', test_rmse, 'train rmse:', train_rmse)
        print('Time:', t, 'seconds')
        print('Time cumulative:', time_cumulative, 'seconds')
        print('.-^-._' * 12)

    for i in range(epoch_f):
        tic = time()
        optimizer_f.minimize(sess, feed_dict={R: train_r})
        pre = sess.run(pred_f, feed_dict={R: train_r})

        t = time() - tic
        time_cumulative += t
        
        error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
        test_rmse = np.sqrt(error)

        error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
        train_rmse = np.sqrt(error_train)

        test_mae = (test_m * np.abs(np.clip(pre, 1., 5.) - test_r)).sum() / test_m.sum()
        train_mae = (train_m * np.abs(np.clip(pre, 1., 5.) - train_r)).sum() / train_m.sum()

        test_ndcg = call_ndcg(np.clip(pre, 1., 5.), test_r)
        train_ndcg = call_ndcg(np.clip(pre, 1., 5.), train_r)

        if test_rmse < best_rmse:
            best_rmse = test_rmse
            best_rmse_ep = i+1

        if test_mae < best_mae:
            best_mae = test_mae
            best_mae_ep = i+1

        if best_ndcg < test_ndcg:
            best_ndcg = test_ndcg
            best_ndcg_ep = i+1

        print('.-^-._' * 12)
        print('FINE-TUNING')
        print('Epoch:', i+1, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
        print('Epoch:', i+1, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
        print('Time:', t, 'seconds')
        print('Time cumulative:', time_cumulative, 'seconds')
        print('.-^-._' * 12)

.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 1 test rmse: 0.886504 train rmse: 0.8716009
Time: 200.3262574672699 seconds
Time cumulative: 200.3262574672699 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 2 test rmse: 0.8534682 train rmse: 0.8168131
Time: 182.8992531299591 seconds
Time cumulative: 383.225510597229 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 3 test rmse: 0.84039575 train rmse: 0.78886414
Time: 186.34759187698364 seconds
Time cumulative: 569.5731024742126 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 4 test rmse: 0.83420444 train rmse: 0.76699173
T

In [ ]:
# Final result
print('Epoch:', best_rmse_ep, ' best rmse:', best_rmse)
print('Epoch:', best_mae_ep, ' best mae:', best_mae)
print('Epoch:', best_ndcg_ep, ' best ndcg:', best_ndcg)

Epoch: 23  best rmse: 0.8229355
Epoch: 16  best mae: 0.6414332
Epoch: 30  best ndcg: 0.9287297571033284


In [ ]:
y_hat

<tf.Tensor 'Sigmoid_5:0' shape=(3706, 500) dtype=float32>

In [ ]:
y

<tf.Tensor 'Sigmoid_3:0' shape=(3706, 500) dtype=float32>

In [ ]:
pre

array([[4.3905616, 4.1434712, 3.8142185, ..., 3.5917108, 4.131508 ,
        3.3984509],
       [3.564693 , 3.1887875, 3.4957132, ..., 3.2197998, 3.205181 ,
        1.59412  ],
       [3.8023353, 3.278904 , 3.3012938, ..., 2.6215024, 2.8641865,
        1.6721148],
       ...,
       [4.041753 , 3.6992843, 3.0517056, ..., 3.5361307, 3.803043 ,
        4.0787   ],
       [4.0346093, 3.49606  , 2.9894578, ..., 3.5343444, 3.6898644,
        4.566532 ],
       [3.917326 , 3.7951741, 3.2021036, ..., 4.0032816, 3.9122841,
        4.233921 ]], dtype=float32)

In [ ]:
pre.shape #movie, user

(3706, 6040)

In [ ]:
len(np.transpose(pre))

6040

In [ ]:
f = open('movie_1m.dat','w')
for i in range (len(np.transpose(pre))):
  for j in range (len(pre)):
    f.write('{}::{}::{} \n '.format(i+1,j+1,round(np.transpose(pre)[i][j])))
f.close()

In [2]:
%pip install neural-tangents
!pip install -r /content/drive/MyDrive/infinite_ac_cf_main/requirements.txt
!pip install sciPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 6.2 MB/s 
     |████████████████████████████████| 110 kB 74.6 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.8 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pre

array([[4.4135394, 4.12291  , 3.2787788, ..., 4.3908305, 4.373799 ,
        3.6966176],
       [2.9125788, 2.9991517, 2.571378 , ..., 3.3772337, 3.6530075,
        3.5007908],
       [3.2719262, 2.693303 , 2.747705 , ..., 3.2841537, 2.8917696,
        3.718302 ],
       ...,
       [2.9467444, 3.2135677, 2.90559  , ..., 3.6273792, 3.8319266,
        2.0522108],
       [2.4696963, 2.8534532, 2.5415885, ..., 3.312544 , 3.673776 ,
        2.3020263],
       [2.7655137, 2.756269 , 2.8474503, ..., 3.2122643, 3.0982168,
        2.6500058]], dtype=float32)

In [ ]:
np.save('GlocalK_weight_m1',pre)

In [ ]:
np.save('/content/drive/MyDrive/GlocalK_weight_m1', pre)

In [5]:
!mkdir -p data/
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -P data/
!cd data/ ; unzip ml-1m.zip ; rm ml-1m.zip ; cd ..

--2022-12-12 03:58:30--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘data/ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.20MB/s    in 1.8s    

2022-12-12 03:58:33 (3.20 MB/s) - ‘data/ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [6]:
%cd /content/drive/MyDrive/infinite_ac_cf_main

/content/drive/MyDrive/infinite_ac_cf_main


In [20]:
import numpy as np

In [23]:
pwd

'/content/drive/MyDrive/infinite_ac_cf_main'

In [32]:
x_save_load = np.load('./test_m.npy')

In [34]:
test = []
for i in range(len(np.transpose(x_save_load))):
  for j in range(len(x_save_load)):
    if np.transpose(x_save_load)[i][j] !=0 :
      test.append([i+1,j+1,1])

In [31]:
import pickle

In [36]:
with open('test.pickle', 'wb') as f:
    pickle.dump(test, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
test

In [37]:
with open('test.pickle', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

In [100]:
#!mkdir -p data/
#!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -P data/
#!cd data/ ; unzip ml-1m.zip ; rm ml-1m.zip ; cd ..
!python preprocess.py ml-1m



!!!!!!!! STARTED PROCESSING ml-1m !!!!!!!!


In [ ]:
!python preprocess.py ml-1m

In [57]:
 d= map(float,test[0])

In [72]:
t = {1,2,3,4}

In [73]:
t = list(t)

In [77]:
for i in range(len(t)):
  t[i] = float(t[i])

In [79]:
t = set(t)

In [80]:
t

{1.0, 2.0, 3.0, 4.0}

In [64]:
for i in range len(t):
  t[i]

4

In [61]:
float(test[0][1])

640.0

In [103]:
!CUDA_VISIBLE_DEVICES=0 python main.py

6040 3706
[{1025, 964, 1848, 2969, 2586, 2557, 253}, {1154, 579, 1414, 1306, 2892, 3341, 1822, 1777, 1012, 1428, 1782, 309, 2426, 702}, {1826, 1788, 2664, 1169, 1781, 538, 1212}, {1025, 1106, 3294, 1111}, {258, 1158, 1546, 3341, 2708, 1430, 279, 38, 1705, 49, 2234, 1603, 3025, 2130, 217, 2651, 3550, 2529, 3042, 1123, 2166}, {1826, 965, 3301, 2859, 3275, 3341, 1837, 942, 3510}, {1445, 1131, 428, 2708, 443}, {513, 3043, 102, 2507, 524, 2956, 1294, 13, 496, 2127, 1587, 2105, 157, 1566, 383}, {1120, 1260, 2931, 2932, 1173, 3029, 339, 24, 2651, 443, 414, 735}, {1281, 1029, 1160, 3083, 268, 2958, 2710, 537, 1307, 1052, 1822, 1823, 1574, 1449, 174, 1455, 957, 3134, 706, 2243, 2884, 1993, 2124, 1741, 2512, 977, 1106, 857, 1114, 1886, 1120, 2785, 3040, 3429, 2667, 1906, 1779, 2162, 246, 1143, 1273, 1915}, {2561, 1186, 2599, 2601, 1707, 2203, 2383, 1295, 1618, 2930, 466, 726, 1017, 923, 991}, {3416, 1131, 1155, 2599}, {407, 3457, 2439, 1449, 1325, 1234, 370, 3186, 1271, 1273, 58, 253}, {2712, 28

In [2]:
print("| end of step  0 | time = 45.46 | HR@10 = 50.0546 | HR@100 = 49.9546 | NDCG@10 = 50.0814 | NDCG@100 = 49.9721 | PSP@10 = 4.9974 | PSP@100 = 0.4991 | AUC = 0.9102 |")

| end of step  0 | time = 45.46 | HR@10 = 50.0546 | HR@100 = 49.9546 | NDCG@10 = 50.0814 | NDCG@100 = 49.9721 | PSP@10 = 4.9974 | PSP@100 = 0.4991 | AUC = 0.9102 |


In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

import time
import copy
import random
import numpy as np

from utils import log_end_epoch, get_item_propensity, get_common_path

def train(hyper_params, data):
    from model import make_kernelized_rr_forward
    from eval import evaluate

    # This just instantiates the function
    kernelized_rr_forward, kernel_fn = make_kernelized_rr_forward(hyper_params)
    sampled_matrix = data.sample_users(hyper_params['user_support']) # Random user sample

    '''
    NOTE: No training required! We will compute dual-variables \alpha on the fly in `kernelized_rr_forward`
          However, if we needed to perform evaluation multiple times, we could pre-compute \alpha like so:
    
    import jax, jax.numpy as jnp, jax.scipy as sp
    @jax.jit
    def precompute_alpha(X, lamda=0.1):
        K = kernel_fn(X, X)
        K_reg = (K + jnp.abs(lamda) * jnp.trace(K) * jnp.eye(K.shape[0]) / K.shape[0])
        return sp.linalg.solve(K_reg, X, sym_pos=True)
    alpha = precompute_alpha(sampled_matrix, lamda=0.1) # Change for the desired value of lamda
    '''

    # Used for computing the PSP-metric
    item_propensity = get_item_propensity(hyper_params, data)
    
    # Evaluation
    start_time = time.time()

    VAL_METRIC = "HR@100"
    best_metric, best_lamda = None, None

    # Validate on the validation-set
    for lamda in [ 0.0, 1.0, 5.0, 20.0, 50.0, 100.0 ] if hyper_params['grid_search_lamda'] else [ hyper_params['lamda'] ]:
        hyper_params['lamda'] = lamda
        val_metrics = evaluate(hyper_params, kernelized_rr_forward, data, item_propensity, sampled_matrix)
        if (best_metric is None) or (val_metrics[VAL_METRIC] > best_metric): best_metric, best_lamda = val_metrics[VAL_METRIC], lamda

    # Return metrics with the best lamda on the test-set
    hyper_params['lamda'] = best_lamda
    test_metrics = evaluate(hyper_params, kernelized_rr_forward, data, item_propensity, sampled_matrix, test_set_eval = True)
    
    log_end_epoch(hyper_params, test_metrics, 0, time.time() - start_time)
    start_time = time.time()

    return test_metrics

def main(hyper_params, gpu_id = None):
    if gpu_id is not None: os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

    from jax.config import config
    if 'float64' in hyper_params and hyper_params['float64'] == True: config.update('jax_enable_x64', True)

    from data import Dataset

    np.random.seed(hyper_params['seed'])
    random.seed(hyper_params['seed'])

    os.makedirs("./results/logs/", exist_ok=True)
    hyper_params['log_file'] = "./results/logs/" + get_common_path(hyper_params) + ".txt"
    
    data = Dataset(hyper_params)
    hyper_params = copy.deepcopy(data.hyper_params) # Updated w/ data-stats

    return train(hyper_params, data)

if __name__ == "__main__":
    from hyper_params import hyper_params
    main(hyper_params)

In [ ]:
import numpy as np
import h5py, sys, os

BASE_PATH = "/"
#base_path

def prep_movielens(ratings_file_path):
    f = open(ratings_file_path, "r")
    users, items, ratings = [], [], []

    line = f.readline()
    while line:
        u, i, r = line.strip().split("::")
        users.append(int(u))
        items.append(int(i))
        ratings.append(float(r))
        line = f.readline()

    min_user = min(users)
    num_users = len(set(users))

    data = [ [] for _ in range(num_users) ]
    for i in range(len(users)):
        data[users[i] - min_user].append([ items[i], ratings[i] ])